In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import re
import io
import matplotlib.pylab as plt
from fa2 import ForceAtlas2
import json
import ast
from collections import Counter
import pickle

In [214]:
import time, sys
from IPython.display import clear_output

#Progress bar
def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

In [215]:
#Open movie data file
df = pd.read_csv('./../../dataset/credits_short.csv', encoding ='utf-8')
number_of_movies = len(df.index)

In [7]:
def get_actors(movie_cast):
    cast_list = []
    for cast_object in movie_cast:
        cast_list.append(cast_object['name'])
    return cast_list


def list_all_actors():
    l = []
    for i, movie_id in enumerate(df['id'], start = 0):
        try:
            cast = df.loc[df['id']==movie_id]['cast'].item()
            cast_objects = ast.literal_eval(cast)
            list_of_actors = get_actors(cast_objects)
            l = l + list_of_actors
        except:
            pass
    return l

def list_actors_in_multiple_movies():
    l = list_all_actors()
    cnt = Counter(l)
    return ([k for k, v in cnt.items() if v > 1])

def get_director(movie_crew):
    for crew_object in movie_crew:
        if (crew_object['job'] == 'Director'):
            return crew_object['name']

In [8]:
# Create director network where directors are connected if they employed the same actors
# Warning! Takes forever to run!
directors_network = nx.Graph()
directors_errors = []

for i, movie_id in enumerate(df['id'], start = 0):
    update_progress(i / number_of_movies)
    try:
        
        # get cast
        cast = df.loc[df['id']==movie_id]['cast'].item()
        cast_objects = ast.literal_eval(cast)
        list_of_actors = get_actors(cast_objects)
        
        # get director
        crew = df.loc[df['id']==movie_id]['crew'].item()
        crew_object = ast.literal_eval(crew)
        director = get_director(crew_object)
        # add director to network
        if not director in directors_network:
            directors_network.add_node(director)
        # find movies who share at least one actor 
        # we only run the loop though movies that we have not already used, 
        # as these will already be in the graph
        for other_movie in df['id'][i+1:]:
            try:
                cast_other_movie = df.loc[df['id']==other_movie]['cast'].item()
                cast_objects_other_movie = ast.literal_eval(cast_other_movie)
                actors = get_actors(cast_objects_other_movie)
                
                other_crew = df.loc[df['id']==other_movie]['crew'].item()
                other_crew_object = ast.literal_eval(other_crew)
                other_director = get_director(other_crew_object)
                # if there are actors in common
                if (set(list_of_actors) & set(actors) and director != other_director):
                    if not other_director in directors_network:
                        directors_network.add_node(other_director)
                    # if edge exists, then update weight
                    if (directors_network.has_edge(other_director, director)):
                        directors_network[other_director][director]['weight'] += 1
                        directors_network[other_director][director]['movies'].append((movie_id,other_movie, set(list_of_actors) & set(actors)))
                    # otherwise add edge
                    else:
                        directors_network.add_edge(other_director, director, weight=1)
                        directors_network[other_director][director]['movies']=[(movie_id,other_movie, set(list_of_actors) & set(actors))]
            except:
                directors_errors.append(['error in other movie with id: '+ (other_movie)])
    except:
        directors_errors.append(['error in movie with id: '+ str(movie_id)])
update_progress(1)

Progress: [####################] 100.0%


In [46]:
nx.get_edge_attributes(directors_network,'movies')

{('John Lasseter', 'Martin Scorsese'): [(862, 524, {'Don Rickles'}),
  (524, 863, {'Don Rickles'}),
  (1578, 49013, {'John Turturro'}),
  (11873, 920, {'Paul Newman'}),
  (11873, 49013, {'John Turturro'}),
  (11873, 13934, {'Paul Newman'}),
  (8649, 920, {'John Goodman'}),
  (10843, 920, {'Cheech Marin'}),
  (10843, 49013, {'Cheech Marin'}),
  (10843, 13934, {'Cheech Marin'}),
  (11324, 49013, {'Emily Mortimer'}),
  (49013, 44826, {'Emily Mortimer'})],
 ('John Lasseter', 'Mike Figgis'): [(862, 451, {'Laurie Metcalf'}),
  (862, 11060, {'Laurie Metcalf'}),
  (451, 863, {'Laurie Metcalf'}),
  (863, 11060, {'Laurie Metcalf'}),
  (47232, 49013, {'Jason Isaacs'})],
 ('John Lasseter', 'Amy Heckerling'): [(862, 9603, {'Wallace Shawn'}),
  (862, 73935, {'Wallace Shawn'}),
  (9603, 863, {'Wallace Shawn'}),
  (863, 73935, {'Wallace Shawn'}),
  (16806, 920, {'Michael Keaton'})],
 ('John Lasseter', 'David Fincher'): [(862, 807, {'R. Lee Ermey'}),
  (807, 9487, {'Kevin Spacey'}),
  (807, 863, {'R. L

In [9]:
nx.write_gpickle(directors_network,"popular_directors_network.gpickle")

In [3]:
directors_network = nx.read_gpickle("popular_directors_network.gpickle")

In [64]:
# Add more attributes to the nodes

director_popularity = pd.read_csv('../../director_popularity.csv', encoding ='utf-8')
metadata = pd.read_csv('../../dataset/movies_metadata_short.csv', encoding ='utf-8')

for node in directors_network.nodes():
    # get director 
    movies = ast.literal_eval(director_popularity.loc[director_popularity['director']==node]['work'].item())
    languages = []
    genres = []
    rating = (director_popularity.loc[director_popularity['director']==node]['vote_average'].item())
    rating_count = (director_popularity.loc[director_popularity['director']==node]['vote_count'].item())
    
    for index, m in enumerate(movies, start = 0):
        release_date = (metadata.loc[metadata['id']==m]['release_date'].item())
        language = (metadata.loc[metadata['id']==m]['original_language'].item())
        languages.append(language)
        movie_genres = ast.literal_eval(metadata.loc[metadata['id']==m]['genres'].item())
        list_of_genres = []
        for g in movie_genres:
            list_of_genres.append(g['name'])
            
        genres += list_of_genres
        
        # initialize to release date
        if index == 0:
            first_movie = release_date
            last_movie = release_date
        else:
            first_movie = min(first_movie, release_date)
            last_movie = max(last_movie, release_date)
        
    directors_network.node[node]['first_movie']  = first_movie
    directors_network.node[node]['last_movie'] = last_movie
    directors_network.node[node]['languages']  = languages
    directors_network.node[node]['genres']  = genres
    directors_network.node[node]['movies']  = movies
    directors_network.node[node]['rating']  = rating
    directors_network.node[node]['rating_count']  = rating_count

In [65]:
nx.write_gpickle(directors_network,"popular_directors_network.gpickle")